# Recreate NEP7 Grid

The goal of this script is to match the NEP7 using functions found within spherical.py and gridUtils.py

In [77]:
import numpy 

# define spherical python functions
def angle_through_center(p1, p2):
    """Angle at center of sphere between two points on the surface of the sphere.
    Positions are given as (latitude,longitude) tuples measured in degrees."""
    phi1 = numpy.deg2rad( p1[0] )
    phi2 = numpy.deg2rad( p2[0] )
    dphi_2 = 0.5 * ( phi2 - phi1 )
    dlambda_2 = 0.5 * numpy.deg2rad( p2[1] - p1[1] )
    a = numpy.sin( dphi_2 )**2 + numpy.cos( phi1 ) * numpy.cos( phi2 ) * ( numpy.sin( dlambda_2 )**2 )
    c = 2. * numpy.arctan2( numpy.sqrt(a), numpy.sqrt( 1. - a ) )
    return c

def quad_area(lat, lon):
    """Returns area of spherical quad (bounded by great arcs)."""
    # x,y,z are 3D coordinates
    d2r = numpy.deg2rad(1.)
    x = numpy.cos(d2r * lat) * numpy.cos(d2r * lon)
    y = numpy.cos(d2r * lat) * numpy.sin(d2r * lon)
    z = numpy.sin(d2r * lat)
    c0 = (x[ :-1, :-1], y[ :-1, :-1], z[ :-1, :-1])
    c1 = (x[ :-1,1:  ], y[ :-1,1:  ], z[ :-1,1:  ])
    c2 = (x[1:  ,1:  ], y[1:  ,1:  ], z[1:  ,1:  ])
    c3 = (x[1:  , :-1], y[1:  , :-1], z[1:  , :-1])
    a0 = angle_between(c1, c0, c2)
    a1 = angle_between(c2, c1, c3)
    a2 = angle_between(c3, c2, c0)
    a3 = angle_between(c0, c3, c1)
    return a0 + a1 + a2 + a3 - 2. * numpy.pi

def angle_between(v1, v2, v3):
    """Returns angle v2-v1-v3 i.e betweeen v1-v2 and v1-v3."""
    # vector product between v1 and v2
    px = v1[1] * v2[2] - v1[2] * v2[1]
    py = v1[2] * v2[0] - v1[0] * v2[2]
    pz = v1[0] * v2[1] - v1[1] * v2[0]
    # vector product between v1 and v3
    qx = v1[1] * v3[2] - v1[2] * v3[1]
    qy = v1[2] * v3[0] - v1[0] * v3[2]
    qz = v1[0] * v3[1] - v1[1] * v3[0]

    ddd = (px * px + py * py + pz * pz) * (qx * qx + qy * qy + qz * qz)
    ddd = (px * qx + py * qy + pz * qz) / numpy.sqrt(ddd)
    angle = numpy.arccos( ddd )
    return angle



In [78]:
# load gridUtils functions

def generate_regional_spherical(lon0, lon_span, lat0, lat_span, tilt, refine):
    """Generate a regional grid centered at (lon0,lat0) with spans of (lon_span,lat_span) and tilted by angle tilt"""
    Ni = int(lon_span*refine)
    Nj = int(lat_span*refine)

    #Generate a mesh at equator centered at (lon0, 0)
    lam_,phi_ = generate_latlon_mesh_centered(Ni,Nj,lon0,lon_span,0.0,lat_span)
    lam_,phi_ = rotate_z_mesh(lam_,phi_, (90.-lon0)*PI_180)  #rotate around z to bring it centered at y axis
    lam_,phi_ = rotate_y_mesh(lam_,phi_,tilt*PI_180)         #rotate around y axis to tilt it as desired
    lam_,phi_ = rotate_x_mesh(lam_,phi_,lat0*PI_180)         #rotate around x to bring it centered at (lon0,lat0)
    lam_,phi_ = rotate_z_mesh(lam_,phi_,-(90.-lon0)*PI_180)  #rotate around z to bring it back

    return lam_,phi_

def rotate_x(x,y,z,theta):
    """Rotate vector (x,y,z) by angle theta around x axis."""
    """Returns the rotated components."""
    cost = np.cos(theta)
    sint = np.sin(theta)
    yp   = y*cost - z*sint
    zp   = y*sint + z*cost
    return x,yp,zp

def rotate_y(x,y,z,theta):
    """Rotate vector (x,y,z) by angle theta around y axis."""
    """Returns the rotated components."""
    cost = np.cos(theta)
    sint = np.sin(theta)
    zp   = z*cost - x*sint
    xp   = z*sint + x*cost
    return xp,y,zp

def rotate_z(x,y,z,theta):
    """Rotate vector (x,y,z) by angle theta around z axis."""
    """Returns the rotated components."""
    cost = np.cos(theta)
    sint = np.sin(theta)
    xp   = x*cost - y*sint
    yp   = x*sint + y*cost
    return xp,yp,z

def pol2cart(lam,phi):
    """Transform a point on globe from Polar (lam,phi) to Cartesian coordinates."""
    """Returns the Cartesian coordinates"""
    lam=lam*PI_180
    phi=phi*PI_180
    x=np.cos(phi)*np.cos(lam)
    y=np.cos(phi)*np.sin(lam)
    z=np.sin(phi)
    return x,y,z
    
def cart2pol(x,y,z):
    """Transform a point on globe from Cartesian (x,y,z) to polar coordinates."""
    """Returns the polar coordinates"""
    lam=np.arctan2(y,x)/PI_180
    phi=np.arctan(z/np.sqrt(x**2+y**2))/PI_180
    return lam,phi

def rotate_z_mesh(lam,phi,theta):
    """Rotate the whole mesh on globe by angle theta around z axis (globe polar axis)."""
    """Returns the rotated mesh."""
    #Bring the angle to be in [-pi,pi] so that atan2 would work
    lam       = np.where(lam>180,lam-360,lam)
    #Change to Cartesian coord
    x,y,z     = pol2cart(lam,phi)
    #Rotate
    xp,yp,zp  = rotate_z(x,y,z,theta)
    #Change back to polar coords using atan2, in [-pi,pi]
    lamp,phip = cart2pol(xp,yp,zp)
    #Bring the angle back to be in [0,2*pi]
    lamp      = np.where(lamp<0,lamp+360,lamp)
    return lamp,phip

def rotate_x_mesh(lam,phi,theta):
    """Rotate the whole mesh on globe by angle theta around x axis (passing through equator and prime meridian.)."""
    """Returns the rotated mesh."""
    #Bring the angle to be in [-pi,pi] so that atan2 would work
    lam       = np.where(lam>180,lam-360,lam)
    #Change to Cartesian coord
    x,y,z     = pol2cart(lam,phi)
    #Rotate
    xp,yp,zp  = rotate_x(x,y,z,theta)
    #Change back to polar coords using atan2, in [-pi,pi]
    lamp,phip = cart2pol(xp,yp,zp)
    #Bring the angle back to be in [0,2*pi]
    lamp      = np.where(lamp<0,lamp+360,lamp)
    return lamp,phip

def rotate_y_mesh(lam,phi,theta):
    """Rotate the whole mesh on globe by angle theta around y axis (passing through equator and prime meridian+90.)."""
    """Returns the rotated mesh."""
    #Bring the angle to be in [-pi,pi] so that atan2 would work
    lam       = np.where(lam>180,lam-360,lam)
    #Change to Cartesian coord
    x,y,z     = pol2cart(lam,phi)
    #Rotate
    xp,yp,zp  = rotate_y(x,y,z,theta)
    #Change back to polar coords using atan2, in [-pi,pi]
    lamp,phip = cart2pol(xp,yp,zp)
    #Bring the angle back to be in [0,2*pi]
    lamp      = np.where(lamp<0,lamp+360,lamp)
    return lamp,phip

def generate_latlon_mesh_centered(lni, lnj, llon0, llen_lon, llat0, llen_lat, ensure_nj_even=True):
    """Generate a regular lat-lon grid"""
    msg = 'Generating regular lat-lon grid centered at %.2f %.2f on equator.' % (llon0, llat0)

    llonSP = llon0 - llen_lon/2 + np.arange(lni+1) * llen_lon/float(lni)
    llatSP = llat0 - llen_lat/2 + np.arange(lnj+1) * llen_lat/float(lnj)
    if(llatSP.shape[0]%2 == 0 and ensure_nj_even):
        llatSP = np.delete(llatSP,0,0)
    llamSP = np.tile(llonSP,(llatSP.shape[0],1))
    lphiSP = np.tile(llatSP.reshape((llatSP.shape[0],1)),(1,llonSP.shape[0]))
    msg = '   Generated regular lat-lon grid between latitudes %.2f %.2f' % (lphiSP[0,0],lphiSP[-1,0])

    msg = '   Number of js=%d' % (lphiSP.shape[0])

    #h_i_inv=llen_lon*self.PI_180*np.cos(lphiSP*self.PI_180)/lni
    #h_j_inv=llen_lat*self.PI_180*np.ones(lphiSP.shape)/lnj
    #delsin_j = np.roll(np.sin(lphiSP*self.PI_180),shift=-1,axis=0) - np.sin(lphiSP*self.PI_180)
    #dx_h=h_i_inv[:,:-1]*self._default_Re
    #dy_h=h_j_inv[:-1,:]*self._default_Re
    #area=delsin_j[:-1,:-1]*self._default_Re*self._default_Re*llen_lon*self.self.PI_180/lni
    return llamSP,lphiSP


### Create Test Values

In [121]:
# define test values
import numpy as np

PI_180 = np.pi/180.
dx = 20
dy = 30

x = np.linspace(300,310,dx)
y = np.linspace(10, 20, dy)
R = 6370.e3 # Radius of sphere        
lon0 = 305
lat0=15
tilt=0
refine=2

# test successful

In [120]:
#dx,dy, x,y,lon0,lat0

### Load NEP7 Grid

In [80]:
# read in nep7 parameters
import xarray as xr

nep7 = xr.open_dataset("/Users/james/Documents/Github/esm_lab/gridtools/nep7_grid/ocean_hgrid.nc")
nep7

<xarray.Dataset>
Dimensions:   (nx: 684, nxp: 685, ny: 1632, nyp: 1633)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    x         (nyp, nxp) float32 ...
    y         (nyp, nxp) float32 ...
    dx        (nyp, nx) float32 ...
    dy        (ny, nxp) float32 ...
    area      (ny, nx) float32 ...
    angle_dx  (nyp, nxp) float32 ...
    tile      |S5 ...
Attributes:
    grid_version:  0.2
    code_version:  ESMG/PyCNAL: heads/python3-0-gb0204bf2e5-dirty
    history:       2020-06-30: convert_ROMS_grid_to_MOM6.py /import/AKWATERS/...

In [127]:
nep7.x.values[-1,:].shape

(685,)

In [171]:
# redefine for nep7 grid

PI_180 = np.pi/180.
dx = len(nep7.x.values[-1,:])
dy = len(nep7.y.values[:,-1])

x = np.linspace(np.min(nep7.x), np.max(nep7.x),dx)
y = np.linspace(np.min(nep7.y), np.max(nep7.y),dy)
R = 6370.e3 # Radius of sphere        
lon0 = np.float(np.abs((np.max(nep7.x) + np.min(nep7.x))) / 2)
lat0=np.float(np.abs((np.max(nep7.y) - np.min(nep7.y))) / 2)
tilt=0
refine=1

In [172]:
#dx,dy, x,y,lon0,lat0

### Make Grid

In [173]:
lonGrid, latGrid = generate_regional_spherical(lon0=lon0, lon_span=dx, lat0=lat0, lat_span=dy, tilt=tilt, refine=refine)
lonGrid

array([[356.67523437, 356.49512248, 356.31661809, ...,   3.68399227,
          3.50548787,   3.32537598],
       [357.35438884, 357.21045134, 357.0676982 , ...,   2.93291215,
          2.79015901,   2.64622152],
       [357.99702056, 357.88762232, 357.77905534, ...,   2.22155502,
          2.11298803,   2.0035898 ],
       ...,
       [182.13443308, 182.25335034, 182.37176218, ..., 177.62884818,
        177.74726002, 177.86617727],
       [182.55553763, 182.6981491 , 182.8401931 , ..., 177.16041726,
        177.30246125, 177.44507273],
       [182.96024794, 183.12569029, 183.29051534, ..., 176.71009501,
        176.87492006, 177.04036241]])

In [174]:
# redfine values for grid metric computation
(nxp, nyp) = lonGrid.shape
grid_x= lonGrid
grid_y = latGrid

### Compute Grid Metrics

In [175]:

            
# Make a copy of the lon grid as values are changed for computation
lon = grid_x.copy()
lat = grid_y
        
        # Approximate edge lengths as great arcs
grid_dx  = R * angle_through_center( (lat[ :,1:],lon[ :,1:]), (lat[:  ,:-1],lon[:  ,:-1]) )
grid_dy = R * angle_through_center( (lat[1:, :],lon[1:, :]), (lat[:-1,:  ],lon[:-1,:  ]) )

# Scaling by latitude?
cos_lat = np.cos(np.radians(lat))

# Presize the angle_dx array
angle_dx = np.zeros(lat.shape)
# Fix lon so they are 0 to 360 for computation of angle_dx
lon = np.where(lon < 0., lon+360, lon)
angle_dx[:,1:-1] = np.arctan2( (lat[:,2:] - lat[:,:-2]) , ((lon[:,2:] - lon[:,:-2]) * cos_lat[:,1:-1]) )
angle_dx[:, 0  ] = np.arctan2( (lat[:, 1] - lat[:, 0 ]) , ((lon[:, 1] - lon[:, 0 ]) * cos_lat[:, 0  ]) )
angle_dx[:,-1  ] = np.arctan2( (lat[:,-1] - lat[:,-2 ]) , ((lon[:,-1] - lon[:,-2 ]) * cos_lat[:,-1  ]) )


area = R * R * quad_area(lat, lon)


In [180]:
lat[:,-1].shape

(1633,)

### Validate

In [153]:
# area has an extra value, hence the -1
area[:,:-1].shape

(1632, 684)

In [147]:
nep7.area.shape

(1632, 684)

In [148]:
area

array([[1.07721477e+09, 1.07721477e+09, 1.07721477e+09, ...,
        1.07721477e+09, 1.07721477e+09, 1.07721477e+09],
       [8.62165659e+08, 8.62165659e+08, 8.62165659e+08, ...,
        8.62165659e+08, 8.62165659e+08, 8.62165659e+08],
       [6.46854040e+08, 6.46854040e+08, 6.46854040e+08, ...,
        6.46854040e+08, 6.46854040e+08, 6.46854040e+08],
       ...,
       [8.62165659e+08, 8.62165659e+08, 8.62165659e+08, ...,
        8.62165659e+08, 8.62165659e+08, 8.62165659e+08],
       [1.07721477e+09, 1.07721477e+09, 1.07721477e+09, ...,
        1.07721477e+09, 1.07721477e+09, 1.07721477e+09],
       [1.29193590e+09, 1.29193590e+09, 1.29193590e+09, ...,
        1.29193590e+09, 1.29193590e+09, 1.29193590e+09]])

In [150]:
nep7.area.values

array([[16991296., 16997952., 17004604., ..., 20201348., 20203766.,
        20206176.],
       [17002518., 17009178., 17015834., ..., 20213982., 20216400.,
        20218810.],
       [17013740., 17020404., 17027064., ..., 20226610., 20229028.,
        20231440.],
       ...,
       [25758958., 25757898., 25756804., ..., 16411003., 16383695.,
        16356341.],
       [25758890., 25757826., 25756730., ..., 16405458., 16378123.,
        16350743.],
       [25758822., 25757756., 25756658., ..., 16399991., 16372631.,
        16345226.]], dtype=float32)

In [158]:
np.min(nep7.area.values)

16345226.0

In [159]:
np.min(area)

-1416347081038.435

# Testing Area


### Shapely & Proj

First stab here is using shapely to calculat the area of a polygon. We would need to alculate the cell corners and then provide the polygon info for this to work. It would likely be too time consuming. To calculate the corners that we need, however, we can use xgcm interp - https://xgcm.readthedocs.io/en/latest/grids.html


In [162]:
co = {"type": "Polygon", "coordinates": [
    [(-102.05, 41.0),
     (-102.05, 37.0),
     (-109.05, 37.0),
     (-109.05, 41.0)]]}
lon, lat = zip(*co['coordinates'][0])
from pyproj import Proj
pa = Proj("+proj=aea +lat_1=37.0 +lat_2=41.0 +lat_0=39.0 +lon_0=-106.55")

x, y = pa(lon, lat)
cop = {"type": "Polygon", "coordinates": [zip(x, y)]}
from shapely.geometry import shape
shape(cop).area  # 268952044107.43506



268952044107.43457

array([[356.67523437, 356.49512248, 356.31661809, ...,   3.68399227,
          3.50548787,   3.32537598],
       [357.35438884, 357.21045134, 357.0676982 , ...,   2.93291215,
          2.79015901,   2.64622152],
       [357.99702056, 357.88762232, 357.77905534, ...,   2.22155502,
          2.11298803,   2.0035898 ],
       ...,
       [182.13443308, 182.25335034, 182.37176218, ..., 177.62884818,
        177.74726002, 177.86617727],
       [182.55553763, 182.6981491 , 182.8401931 , ..., 177.16041726,
        177.30246125, 177.44507273],
       [182.96024794, 183.12569029, 183.29051534, ..., 176.71009501,
        176.87492006, 177.04036241]])

# Geometry Calcs

We started with the formula for the area of the earth between a line of latitude and the north pole (the area of a spherical cap, listed in the Dr. Math FAQ on Geometric Formulas).

A = 2*pi*R*h

where R is the radius of the earth and h is the perpendicular distance from the plane containing the line of latitude to the pole. We can calculate h using trigonometry as

h = R*(1-sin(lat))

Thus the area north of a line of latitude is

A = 2*pi*R^2(1-sin(lat))

The area between two lines of latitude is the difference between the area north of one latitude and the area north of the other latitude:

A = |2*pi*R^2(1-sin(lat2)) - 2*pi*R^2(1-sin(lat1))|
= 2*pi*R^2 |sin(lat1) - sin(lat2)|

The area of a lat-long rectangle is proportional to the difference in the longitudes. The area I just calculated is the area between longitude lines differing by 360 degrees. Therefore the area we seek is

A = 2*pi*R^2 |sin(lat1)-sin(lat2)| |lon1-lon2|/360
= (pi/180)R^2 |sin(lat1)-sin(lat2)| |lon1-lon2|



### What it means
If we use corners with this calculation instead of cell center, we can likely calculate the area with this equation. Thus, an avenue to try here would be calculate the corners with xgcm interp, use the longitude corners as the edge values and the latitude interpolation as the north/south edge values and calculate hte area with teh initial lat/lon as center point?

# Pyproj & Shapely

Go from rectangular grid to projected grid

In [192]:
lonGrid.shape

(1633, 686)

In [12]:
import pyproj
p = pyproj.Proj("+proj=lcc +lon_0=-91 +lat_1=40 +lat_2=60")
p.srs


'+proj=lcc +lat_0=0 +lon_0=-91 +lat_1=40 +lat_2=60 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

In [199]:
np.min(lon)

-90.99999994894988

# Just use rioxarray!

Just as long as it will play nicely, we can use rioxarray to reproject lat/lons - https://corteva.github.io/rioxarray/stable/examples/reproject.html

In [21]:
xr.open_dataset("/Users/james/Downloads/PLANET_SCOPE_3D.nc")


<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
    spatial_ref  int64 ...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * time         (time) datetime64[ns] 2016-12-19T10:27:29.687763 2016-12-29T...
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...

In [6]:
import rioxarray
import xarray as xr

nep7 = xr.open_dataset("/Users/james/Documents/Github/esm_lab/gridtools/nep7_grid/ocean_hgrid.nc")
nep7

<xarray.Dataset>
Dimensions:   (nx: 684, nxp: 685, ny: 1632, nyp: 1633)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    x         (nyp, nxp) float32 ...
    y         (nyp, nxp) float32 ...
    dx        (nyp, nx) float32 ...
    dy        (ny, nxp) float32 ...
    area      (ny, nx) float32 ...
    angle_dx  (nyp, nxp) float32 ...
    tile      |S5 ...
Attributes:
    grid_version:  0.2
    code_version:  ESMG/PyCNAL: heads/python3-0-gb0204bf2e5-dirty
    history:       2020-06-30: convert_ROMS_grid_to_MOM6.py /import/AKWATERS/...

In [81]:
import xarray as xr
import numpy as np

# load nep7
nep7 = xr.open_dataset("/Users/james/Documents/Github/esm_lab/gridtools/nep7_grid/ocean_hgrid.nc")
#nep7 = nep7.assign_coords({"nx": nep7['dx'][-1,].values})

# grab lat/lon values from nep7  grid
lon = nep7.x[-1,].values
lat = nep7.y[:,-1].values

# create xarray object with these lat/lons as the grid cell centers
ds = xr.Dataset(coords={'y_c': (['y_c',], lat),'x_c': (['x_c',], lon)})

# create blank array
ds['var'] = (('y_c', 'x_c'), np.zeros((len(lat), len(lon))))

# set spatial dimensions for rioxarray
ds = ds.rio.set_spatial_dims(y_dim='y_c', x_dim='x_c', inplace=True)


In [82]:
# write the rio crs for rioxarray
ds = ds.rio.write_crs("+proj=lcc +lat_0=0 +lon_0=-91 +lat_1=40 +lat_2=60 +x_0=0 +y_0=0 +datum=WGS84 +no_defs")

ds.spatial_ref



<xarray.DataArray 'spatial_ref' ()>
array(0)
Coordinates:
    spatial_ref  int64 0
Attributes: (12/17)
    crs_wkt:                        PROJCS["unknown",GEOGCS["unknown",DATUM["...
    semi_major_axis:                6378137.0
    semi_minor_axis:                6356752.314245179
    inverse_flattening:             298.257223563
    reference_ellipsoid_name:       WGS 84
    longitude_of_prime_meridian:    0.0
    ...                             ...
    standard_parallel:              (40.0, 60.0)
    latitude_of_projection_origin:  0.0
    longitude_of_central_meridian:  -91.0
    false_easting:                  0.0
    false_northing:                 0.0
    spatial_ref:                    PROJCS["unknown",GEOGCS["unknown",DATUM["...

In [83]:
xds_lonlat = ds.rio.reproject("+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext  +no_defs")

In [84]:
xds_lonlat

<xarray.Dataset>
Dimensions:      (x: 283, y: 1748)
Coordinates:
  * x            (x) float64 -1.013e+07 -1.013e+07 ... -1.013e+07 -1.013e+07
  * y            (y) float64 59.1 59.07 59.05 59.02 ... 14.92 14.9 14.87 14.84
    spatial_ref  int64 0
Data variables:
    var          (y, x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    grid_mapping:  spatial_ref

In [70]:
xds_lonlat.y.values

array([59.09620419, 59.07087413, 59.04554406, ..., 14.89523774,
       14.86990767, 14.84457761])

In [ ]:
# create nep7 grid from epsg projection

In [85]:
# load nep7
nep7 = xr.open_dataset("/Users/james/Documents/Github/esm_lab/gridtools/nep7_grid/ocean_hgrid.nc")
#nep7 = nep7.assign_coords({"nx": nep7['dx'][-1,].values})

# grab lat/lon values from nep7  grid
lon = nep7.x[-1,].values
lat = nep7.y[:,-1].values

# create xarray object with these lat/lons as the grid cell centers
ds = xr.Dataset(coords={'y_c': (['y_c',], lat),'x_c': (['x_c',], lon)})

# create blank array
ds['var'] = (('y_c', 'x_c'), np.zeros((len(lat), len(lon))))

# set spatial dimensions for rioxarray
ds = ds.rio.set_spatial_dims(y_dim='y_c', x_dim='x_c', inplace=True)

# write the rio crs for rioxarray
ds = ds.rio.write_crs("EPSG:4326")

ds.spatial_ref


<xarray.DataArray 'spatial_ref' ()>
array(0)
Coordinates:
    spatial_ref  int64 0
Attributes:
    crs_wkt:                      GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["...
    semi_major_axis:              6378137.0
    semi_minor_axis:              6356752.314245179
    inverse_flattening:           298.257223563
    reference_ellipsoid_name:     WGS 84
    longitude_of_prime_meridian:  0.0
    prime_meridian_name:          Greenwich
    geographic_crs_name:          WGS 84
    grid_mapping_name:            latitude_longitude
    spatial_ref:                  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["...

In [86]:
xds_lonlat = ds.rio.reproject("+proj=lcc +lat_0=0 +lon_0=-91 +lat_1=40 +lat_2=60 +x_0=0 +y_0=0 +datum=WGS84 +no_defs")
xds_lonlat

<xarray.Dataset>
Dimensions:      (x: 1770, y: 399)
Coordinates:
  * x            (x) float64 -8.623e+06 -8.619e+06 ... -1.622e+06 -1.618e+06
  * y            (y) float64 1.129e+07 1.129e+07 ... 9.717e+06 9.713e+06
    spatial_ref  int64 0
Data variables:
    var          (y, x) float64 -9.999e+03 -9.999e+03 ... -9.999e+03 -9.999e+03
Attributes:
    grid_mapping:  spatial_ref